Updating stations in OSCAR/Surface
---
> **Example #3:** We will update the schedules of German stations. (A) All stations that have been edited _in the last two years_ ***and*** have a _1h reporting interval_ ***as well as*** a _30 minute reporting interval_ should have the international exchange flag set to _true_ for the _30 min interval_ and to _false_ for the _1h interval_.  (B) Stations that only have a _30 min reporting interval_ should have this reporting interval changed to 1h and the international exchange flag set to _true_.